# 1.Import related modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/resnet_cifar10

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/resnet_cifar10


In [ ]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import d2lzh_pytorch as d2l
%matplotlib inline

# 2.Obtain the data set and do preprocessing

## Preparing the Data


In [ ]:
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
os.listdir('cifar10/cifar10')

['test', 'labels.txt', 'train']

In [ ]:

data_dir = 'cifar10/cifar10'
print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['test', 'labels.txt', 'train']
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [ ]:
# Data transforms (normalization & data augmentation)
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
train_tfms = tt.Compose([tt.RandomCrop(32, padding=4, padding_mode='reflect'),
                         tt.RandomHorizontalFlip(),
                         tt.ToTensor(),
                         tt.Normalize(*stats,inplace=True)])
valid_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])

In [ ]:
# PyTorch datasets
cifar10_train  = ImageFolder(data_dir+'/train', train_tfms)
cifar10_test = ImageFolder(data_dir+'/test', valid_tfms)


Preprocessing and transforming images can improve model accuracy and convergence speed.

In [ ]:
# Print the data type of the training dataset
print(type(cifar10_train))

# Print the number of images in the training and test datasets
print(len(cifar10_train), len(cifar10_test))


<class 'torchvision.datasets.cifar.CIFAR10'>
50000 10000


In [ ]:
# Retrieve a sample from the training dataset (the fourth image)
feature, label = cifar10_train[3]

# Print the dimensions and data type of the image
print(feature.shape, feature.dtype)

# Print the label associated with the image
print(label)


torch.Size([3, 32, 32]) torch.float32
4


In [ ]:
def get_labels(numeric_labels):
    # List of human-readable labels for CIFAR-10 categories
    text_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']
    # Map each numeric label to its corresponding text label
    return [text_labels[int(label)] for label in numeric_labels]

print(get_labels([0,1,2,3,4,5,6,7,8,9]))


['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


# 3. Define ResNet18 model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Adjust shortcut to match dimension changes caused by convolutions
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))  # Activation after first convolution
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)               # Add output of shortcut to main path
        out = F.relu(out)                     # Final activation after addition
        return out


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64  # Initial number of channels

        # Initial convolution and batch normalization
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        # Create each ResNet layer with varying number of blocks
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        # Fully connected layer for classification
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    # Helper function to create layers of blocks
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, planes, stride))
            self.in_channels = planes * block.expansion
        return nn.Sequential(*layers)

    # Forward propagation function
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))  # Activation after initial convolution
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)  # Average pooling
        out = out.view(out.size(0), -1)  # Flatten
        out = self.linear(out)  # Fully connected layer
        return out


In [ ]:
# Initialize ResNet with BasicBlock and 2 blocks in each of its 4 layers
net = ResNet(BasicBlock, [2, 2, 2, 2])

print(net)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

# 4. Training model preparation

In [ ]:
# Set batch size for training
batch_size = 128

# Initialize data loaders for training and testing
train_iter = torch.utils.data.DataLoader(cifar10_train, batch_size=batch_size, shuffle=True, num_workers=2)
test_iter = torch.utils.data.DataLoader(cifar10_test, batch_size=100, shuffle=False, num_workers=2)


In [ ]:
# Enable multi-GPU training if CUDA is available
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True


An accuracy evaluation function is created, which is mainly used to input the network model obtained in each round into the test set data during the training process to obtain its generalization accuracy.


In [ ]:
# Evaluate model accuracy on a dataset
import torch

def calculate_accuracy(data_loader, model, device=None):
    if device is None and isinstance(model, torch.nn.Module):
        device = next(model.parameters()).device

    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            model.eval()
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            model.train()

            total_samples += labels.size(0)

    return correct_predictions / total_samples



Created a training function, mainly used for loop training, and after each round of training, print the training set accuracy and test set accuracy of this round and other training information

In [ ]:
import torch
import time

def train(model, train_loader, test_loader, batch_size, optimizer, device, num_epochs):
    model = model.to(device)
    print("Training on", device)
    criterion = torch.nn.CrossEntropyLoss()
    batch_count = 0

    for epoch in range(num_epochs):
        train_loss_sum, train_acc_sum, n, start_time = 0.0, 0.0, 0, time.time()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss_sum += loss.item()
            train_acc_sum += (outputs.argmax(dim=1) == labels).sum().item()
            n += labels.size(0)
            batch_count += 1

        test_acc = calculate_accuracy(test_loader, model, device)
        print('Epoch %d, Loss: %.4f, Train Acc: %.3f, Test Acc: %.3f, Time: %.1f sec'
              % (epoch + 1, train_loss_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start_time))


# 5.Training model

In [ ]:
# Set learning rate and number of epochs
lr, num_epochs = 0.01, 50
# Initialize the optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
# Start training the model
train(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on cuda
epoch 1, loss 1.8015, train acc 0.335, test acc 0.419, time 391.6 sec
epoch 2, loss 0.6479, train acc 0.530, test acc 0.599, time 382.3 sec
epoch 3, loss 0.3319, train acc 0.638, test acc 0.661, time 381.4 sec
epoch 4, loss 0.2107, train acc 0.705, test acc 0.732, time 381.9 sec
epoch 5, loss 0.1421, train acc 0.753, test acc 0.753, time 383.0 sec
epoch 6, loss 0.1010, train acc 0.790, test acc 0.780, time 383.1 sec
epoch 7, loss 0.0753, train acc 0.817, test acc 0.804, time 381.5 sec
epoch 8, loss 0.0582, train acc 0.839, test acc 0.824, time 381.4 sec
epoch 9, loss 0.0473, train acc 0.853, test acc 0.842, time 381.7 sec
epoch 10, loss 0.0390, train acc 0.865, test acc 0.837, time 381.8 sec
epoch 11, loss 0.0325, train acc 0.879, test acc 0.857, time 382.8 sec
epoch 12, loss 0.0275, train acc 0.885, test acc 0.846, time 382.0 sec
epoch 13, loss 0.0240, train acc 0.894, test acc 0.850, time 382.8 sec
epoch 14, loss 0.0208, train acc 0.901, test acc 0.865, time 381.7 se